In [1]:
import dlc_bci as bci

In [2]:
train_input, train_target = bci.load(root = './data_bci')
test_input, test_target = bci.load(root = './data_bci', train = False)
print(str(type(train_input)), train_input.size()) 
print(str(type(train_target)), train_target.size())
print(str(type(test_input)), test_input.size()) 
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])


In [3]:
import torch
from torch import cuda , nn , optim 
from torch.nn import functional as F 
from torch.autograd import Variable 

In [4]:
# Normalize
mu, std = train_input.mean(), train_input.std()
train_input.sub_(mu).div_(std)
test_input.sub_(mu).div_(std)
# convert to Variable
train_input, train_target = Variable(train_input), Variable(train_target)
test_input, test_target = Variable(test_input), Variable(test_target)

## Support Vector Machine

In [10]:
import numpy as np
from sklearn import svm 

In [9]:
X_train = train_input.view(316,-1)
y_train = train_target
X_test = test_input.view(100,-1)
y_test = test_target

In [11]:
clf = svm.SVC(kernel='linear')
clf.fit(X_train.data, y_train.data)
clf.score(X_test.data, y_test.data)

0.75

# Neural Network

In [8]:
train_input = train_input.view(316,1,50,28)
test_input = test_input.view(100,1,50,28)

In [9]:
print(str(type(train_input)), train_input.size()) 
print(str(type(train_target)), train_target.size())
print(str(type(test_input)), test_input.size()) 
print(str(type(test_target)), test_target.size())

<class 'torch.autograd.variable.Variable'> torch.Size([316, 1, 50, 28])
<class 'torch.autograd.variable.Variable'> torch.Size([316])
<class 'torch.autograd.variable.Variable'> torch.Size([100, 1, 50, 28])
<class 'torch.autograd.variable.Variable'> torch.Size([100])


In [45]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 30, kernel_size=(7,1))
        self.conv1_drop = nn.Dropout(0.3)
        self.conv2 = nn.Conv2d(30, 10, kernel_size=(5,5))
        self.conv2_drop = nn.Dropout(0.3)
        self.conv3 = nn.Conv2d(10, 5, kernel_size=(1,1))
        self.conv3_drop = nn.Dropout(0.3)
        self.fc1 = nn.Linear(1200,120)
        self.fc1_drop = nn.Dropout(0.3)
        self.fc2 = nn.Linear(120,100)
        self.fc2_drop = nn.Dropout(0.3)
        self.fc3 = nn.Linear(100,2)
    def forward(self, x): 
        x = F.relu(self.conv1(x))
        x = self.conv1_drop(x)
        x = F.relu(self.conv2(x))
        x = self.conv2_drop(x)
        x = F.relu(self.conv3(x))
        x = self.conv3_drop(x)
        
        x = F.max_pool2d(x, (2,2))
        
        x = x.view(x.size(0),-1)
       
        x = F.relu(self.fc1(x))
        x = self.fc1_drop(x)
        x = F.relu(self.fc2(x))
        x = self.fc2_drop(x)
        x = F.relu(self.fc3(x))
        return x

In [32]:
def train_model(model, train_input, train_target):
    lr, nb_epochs = 1e-1, 10 
    optimizer = optim.SGD(model.parameters(), lr = lr)
    criterion = torch.nn.CrossEntropyLoss()
    for k in range(nb_epochs):
        optimizer.zero_grad()
        output = model(train_input) 
        loss = criterion(output, train_target) 
        loss.backward()
        optimizer.step()

In [33]:
def compute_nb_errors(model, data_input, data_target):
    nb_errors = 0
    output = model.forward(data_input)
    pred = output.data.max(1, keepdim=True)[1]
    nb_errors = pred.ne(data_target.data.view_as(pred)).cpu().sum()
    return nb_errors

In [43]:
model = Net()

In [35]:
train_model(model, train_input, train_target)

/Users/suntao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [36]:
nb_error = compute_nb_errors(model, test_input, test_target)

/Users/suntao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [37]:
print('Error Rate: {}'.format(nb_error/test_input.size(0)))

Error Rate: 0.49
